In [1]:
import pyodbc

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

CODIGO PARA PODER RECUPERAR LOS DATOS

In [3]:
# Datos de conexión
server = 'practicasai.database.windows.net'
database = 'adminibm'
username = 'profesor01'
password = 'Profesor!123'
driver = '{ODBC Driver 18 for SQL Server}'

In [4]:
# Crear conexión
conn = pyodbc.connect(f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;')

Las tres tablas que se pueden consultar en la nube son 
- FactInventory
- FactOnlineSales
- FactSales

In [8]:
df_FactInventory = pd.read_sql_query("SELECT TOP 1000 * FROM FactInventory", conn)

C:\Users\erick\AppData\Local\Temp\ipykernel_12140\1797492992.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_FactInventory = pd.read_sql_query("SELECT TOP 1000 * FROM FactInventory", conn)


In [9]:
df_FactOnlineSales = pd.read_sql_query("SELECT TOP 1000 * FROM FactOnlineSales", conn)

C:\Users\erick\AppData\Local\Temp\ipykernel_12140\1302041542.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_FactOnlineSales = pd.read_sql_query("SELECT TOP 1000 * FROM FactOnlineSales", conn)


In [10]:
df_FactSales = pd.read_sql_query("SELECT TOP 1000 * FROM FactSales", conn)

C:\Users\erick\AppData\Local\Temp\ipykernel_12140\1491214618.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_FactSales = pd.read_sql_query("SELECT TOP 1000 * FROM FactSales", conn)


In [ ]:
df_DimCustomer = pd.read_csv("C:\\Nicolas\\Proyecto Medio Ambiental\\Proyecto_Guayerd\\Guayerd_InteligenciaArtificial\\Datos\\DimCustomer.csv")

In [7]:
df_DimProduct = pd.read_csv("C:\\Nicolas\\Proyecto Medio Ambiental\\Proyecto_Guayerd\\Guayerd_InteligenciaArtificial\\Datos\\DimProduct.csv")

In [22]:
# ---------- 1) Normalizar fechas ----------
for df, col in [(df_FactInventory, 'DateKey'),
                (df_FactOnlineSales, 'DateKey'),
                (df_FactSales, 'DateKey')]:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')



In [23]:
# ---------- 2) Merge con DimProduct (campos útiles) ----------
prod_cols_guess = ['ProductKey','ProductLabel','ProductName','ProductSubcategoryName','UnitCost','UnitPrice']
prod_cols = [c for c in prod_cols_guess if c in df_DimProduct.columns]

if 'ProductKey' not in prod_cols:
    raise RuntimeError("DimProduct debe contener ProductKey.")

df_prod_small = df_DimProduct[prod_cols].drop_duplicates('ProductKey')

df_store = df_FactSales.merge(df_prod_small, on='ProductKey', how='left', suffixes=('','_prod'))
df_online = df_FactOnlineSales.merge(df_prod_small, on='ProductKey', how='left', suffixes=('','_prod'))



In [ ]:
# ---------- 1) Encabezado y columnas presentes ----------
def analyze_channel_header(df, channel_name):
    print("\n" + "="*70)
    print(f"ANÁLISIS CANAL: {channel_name}")
    print("="*70)

    # Columnas de interés
    cols_interest = ['SalesQuantity','SalesAmount','UnitPrice','UnitCost','TotalCost','OnHandQuantity','DaysInStock','PromotionKey']
    cols_present = [c for c in cols_interest if c in df.columns]
    print("Columnas analizadas (presentes):", cols_present)
    return cols_present



In [50]:
# ---------- 2) Estadísticas descriptivas y top productos ----------
def analyze_channel_describe(df):
    # Estadísticas descriptivas
    numeric_display = [c for c in ['SalesQuantity','SalesAmount','UnitPrice','UnitCost'] if c in df.columns]
    if numeric_display:
        print("\nEstadísticas descriptivas (describe):")
        try:
            print(df[numeric_display].describe(percentiles=[0.01,0.05,0.25,0.5,0.75,0.95,0.99]).T.to_string())
        except Exception:
            print(df[numeric_display].describe().T.to_string())
    else:
        print("No hay columnas numéricas típicas para describir.")

    # Top productos por ventas
    if 'ProductKey' in df.columns and 'SalesAmount' in df.columns:
        top_by_sales = df.groupby('ProductKey', dropna=False).agg(
            SalesAmount=('SalesAmount','sum'),
            SalesQuantity=('SalesQuantity','sum')
        ).sort_values('SalesAmount', ascending=False).head(10)
        print("\nTop 10 productos por SalesAmount:")
        print(top_by_sales.to_string())
    else:
        top_by_sales = None
    return top_by_sales

In [49]:
# ---------- 3) Distribución y normalidad ----------
from scipy import stats

def analyze_channel_distribution(df):
    import pandas as pd
    from scipy import stats
    import numpy as np

    def distrib_info(series):
        s = series.dropna()
        if s.empty:
            return None
        skew = float(s.skew())
        kurt = float(s.kurtosis())
        normal_test = {}
        try:
            if len(s) <= 5000:
                st, p = stats.shapiro(s.sample(min(len(s),5000)))
                normal_test = {'test':'shapiro','stat':float(st),'p':float(p)}
            else:
                z = (s - s.mean())/s.std(ddof=0)
                st, p = stats.kstest(z, 'norm')
                normal_test = {'test':'kstest_vs_normal','stat':float(st),'p':float(p)}
        except:
            pass
        return {
            'count': len(s),
            'mean': s.mean(),
            'median': s.median(),
            'std': s.std(),
            'skew': skew,
            'kurtosis': kurt,
            'normal_test_p': normal_test.get('p', np.nan)
        }

    results = []
    for col in ['SalesAmount','SalesQuantity','UnitPrice','UnitCost']:
        if col in df.columns:
            info = distrib_info(df[col])
            if info is not None:
                info['column'] = col
                results.append(info)

    if results:
        df_res = pd.DataFrame(results)
        df_res = df_res[['column','count','mean','median','std','skew','kurtosis','normal_test_p']]
        print("\nDistribución de columnas (estética):")
        print(df_res.round(3).to_string(index=False))
    else:
        print("No hay columnas numéricas para mostrar.")

In [48]:
# ---------- 4) Correlaciones ----------
def analyze_channel_correlations(df):
    corr_candidates = [c for c in ['UnitPrice','UnitCost','SalesQuantity','SalesAmount','TotalCost','OnHandQuantity','DaysInStock'] if c in df.columns]
    if len(corr_candidates) >= 2:
        corr_df = df[corr_candidates].dropna()
        if not corr_df.empty:
            print("\nCorrelación Pearson:")
            print(corr_df.corr(method='pearson').round(3).to_string())
            print("\nCorrelación Spearman:")
            print(corr_df.corr(method='spearman').round(3).to_string())
        else:
            print("No hay suficientes filas no-NA para correlaciones.")
    else:
        print("No hay suficientes columnas numéricas para correlación en este canal.")

In [47]:
# ---------- 5) Outliers ----------
def analyze_channel_outliers(df):
    print("\nDetección de outliers (IQR) -> SalesAmount, SalesQuantity, UnitPrice:")
    outlier_examples = {}
    for col in ['SalesAmount','SalesQuantity','UnitPrice']:
        if col in df.columns:
            s = df[col].dropna()
            if len(s) == 0:
                continue
            Q1 = s.quantile(0.25); Q3 = s.quantile(0.75); IQR = Q3 - Q1
            low = Q1 - 1.5*IQR; high = Q3 + 1.5*IQR
            mask = (df[col] < low) | (df[col] > high)
            n_out = int(mask.sum())
            print(f" {col}: {n_out} outliers. Ejemplos:")
            ex = df.loc[mask, ['ProductKey', col, 'DateKey']].head(5)
            if not ex.empty:
                print(ex.to_string(index=False))
            outlier_examples[col] = ex
    return outlier_examples

In [51]:
# ---------- 4) Ejecutar análisis por separado ----------
cols = analyze_channel_header(df_store, "TIENDA")


ANÁLISIS CANAL: TIENDA
Columnas analizadas (presentes): ['SalesQuantity', 'SalesAmount', 'UnitPrice', 'UnitCost', 'TotalCost', 'PromotionKey']


In [52]:
top_products = analyze_channel_describe(df_store)



Estadísticas descriptivas (describe):
                count         mean          std     min         1%         5%         25%        50%         75%         95%           99%         max
SalesQuantity  1000.0    14.559000    24.036853  2.0000   3.000000    4.00000    9.000000    10.0000    13.00000    30.00000     90.100000    480.0000
SalesAmount    1000.0  2674.151505  3814.160193  8.0282  42.153976  109.09118  488.704200  1619.6479  3380.28170  8902.77284  17732.821594  52923.4155
UnitPrice      1000.0   233.111119   305.497628  0.6690   3.792983    9.78835   54.867925   140.8380   281.68485   703.55279   1760.936664   2253.5141
UnitCost       1000.0    98.181184   113.747320  0.3380   1.935121    4.66900   25.429600    61.0423   129.53520   303.08450    618.774600    746.6338

Top 10 productos por SalesAmount:
            SalesAmount  SalesQuantity
ProductKey                            
2085         69335.3873           68.0
1857         46705.9859           25.0
380          30

In [45]:
analyze_channel_distribution(df_store)


Distribución de columnas (estética):
       column  count     mean   median      std   skew  kurtosis  normal_test_p
  SalesAmount   1000 2674.152 1619.648 3814.160  4.961    43.021            0.0
SalesQuantity   1000   14.559   10.000   24.037 13.114   218.472            0.0
    UnitPrice   1000  233.111  140.838  305.498  3.238    13.468            0.0
     UnitCost   1000   98.181   61.042  113.747  2.649     8.859            0.0


In [36]:
analyze_channel_correlations(df_store)


Correlación Pearson:
               UnitPrice  UnitCost  SalesQuantity  SalesAmount  TotalCost
UnitPrice          1.000     0.977         -0.092        0.758      0.719
UnitCost           0.977     1.000         -0.102        0.739      0.738
SalesQuantity     -0.092    -0.102          1.000        0.061      0.076
SalesAmount        0.758     0.739          0.061        1.000      0.975
TotalCost          0.719     0.738          0.076        0.975      1.000

Correlación Spearman:
               UnitPrice  UnitCost  SalesQuantity  SalesAmount  TotalCost
UnitPrice          1.000     0.992         -0.023        0.916      0.902
UnitCost           0.992     1.000         -0.020        0.911      0.910
SalesQuantity     -0.023    -0.020          1.000        0.323      0.339
SalesAmount        0.916     0.911          0.323        1.000      0.992
TotalCost          0.902     0.910          0.339        0.992      1.000


In [37]:
outliers = analyze_channel_outliers(df_store)


Detección de outliers (IQR) -> SalesAmount, SalesQuantity, UnitPrice:
 SalesAmount: 63 outliers. Ejemplos:
 ProductKey  SalesAmount    DateKey
       2365    9858.9085 2007-07-02
       1946    7723.8028 2008-12-08
        202    8661.0211 2009-03-04
        564   15750.7394 2007-01-12
       2085   10387.3239 2007-10-18
 SalesQuantity: 165 outliers. Ejemplos:
 ProductKey  SalesQuantity    DateKey
       1133           24.0 2008-01-22
       2365           36.0 2007-07-02
       1731           24.0 2008-07-14
         47           20.0 2009-07-17
         49           26.0 2008-12-23
 UnitPrice: 73 outliers. Ejemplos:
 ProductKey  UnitPrice    DateKey
       1946   633.0986 2008-12-08
        564  1759.8592 2007-01-12
        341   680.9859 2009-10-30
       1866   704.1549 2007-04-17
       2085  1038.7324 2007-10-18


In [ ]:
# ---------- 1) Encabezado y columnas presentes ----------
cols_online = analyze_channel_header(df_online, "ONLINE (FactOnlineSales)")


ANÁLISIS CANAL: ONLINE (FactOnlineSales)
Columnas analizadas (presentes): ['SalesQuantity', 'SalesAmount', 'UnitPrice', 'UnitCost', 'TotalCost', 'PromotionKey']


In [ ]:
# ---------- 2) Estadísticas descriptivas y top productos ----------
top_products_online = analyze_channel_describe(df_online)


In [46]:
# ---------- 3) Distribución y normalidad ----------
analyze_channel_distribution(df_online)


Distribución de columnas (estética):
       column  count    mean  median     std   skew  kurtosis  normal_test_p
  SalesAmount   1000  85.078  20.276 158.788  4.845    30.183            0.0
SalesQuantity   1000   0.983   1.000   0.129 -7.484    54.117            0.0
    UnitPrice   1000 108.315  29.387 198.655  4.811    29.889            0.0
     UnitCost   1000  46.354  14.986  79.372  4.391    24.738            0.0


In [41]:
# ---------- 4) Correlaciones ----------
analyze_channel_correlations(df_online)


Correlación Pearson:
               UnitPrice  UnitCost  SalesQuantity  SalesAmount  TotalCost
UnitPrice          1.000     0.973         -0.005        0.994      0.973
UnitCost           0.973     1.000          0.001        0.969      1.000
SalesQuantity     -0.005     0.001          1.000        0.070      0.001
SalesAmount        0.994     0.969          0.070        1.000      0.969
TotalCost          0.973     1.000          0.001        0.969      1.000

Correlación Spearman:
               UnitPrice  UnitCost  SalesQuantity  SalesAmount  TotalCost
UnitPrice          1.000     0.997         -0.035        0.959      0.997
UnitCost           0.997     1.000         -0.032        0.957      1.000
SalesQuantity     -0.035    -0.032          1.000        0.225     -0.032
SalesAmount        0.959     0.957          0.225        1.000      0.957
TotalCost          0.997     1.000         -0.032        0.957      1.000


In [40]:
# ---------- 5) Outliers ----------
outliers_online = analyze_channel_outliers(df_online)


Detección de outliers (IQR) -> SalesAmount, SalesQuantity, UnitPrice:
 SalesAmount: 40 outliers. Ejemplos:
 ProductKey  SalesAmount    DateKey
        599    1292.9577 2007-01-01
        599    1292.9577 2007-01-01
        599    1292.9577 2007-01-01
        599    1292.9577 2007-01-01
        599    1292.9577 2007-01-01
 SalesQuantity: 17 outliers. Ejemplos:
 ProductKey  SalesQuantity    DateKey
       2506            0.0 2007-01-01
       1484            0.0 2007-01-01
       1558            0.0 2007-01-01
       1819            0.0 2007-01-01
       1488            0.0 2007-01-01
 UnitPrice: 41 outliers. Ejemplos:
 ProductKey  UnitPrice    DateKey
        599  1616.1972 2007-01-01
        599  1616.1972 2007-01-01
        599  1616.1972 2007-01-01
        599  1616.1972 2007-01-01
        599  1616.1972 2007-01-01
